# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 31 2022 10:07AM,238395,18,CLRSPC-21024744,"ClearSpec, LLC",Released
1,May 31 2022 10:07AM,238393,18,8086566,"Miracle Fruit Oil, LLC",Released
2,May 31 2022 10:07AM,238394,19,ELI53122-3,Eli Lilly and Company,Released
3,May 31 2022 10:06AM,238322,15,VT80172946,"Virtus Pharmaceuticals, LLC",Released
4,May 31 2022 10:06AM,238322,15,VT80173031,"Virtus Pharmaceuticals, LLC",Released
5,May 31 2022 10:06AM,238322,15,VT80173032,"Virtus Pharmaceuticals, LLC",Released
6,May 31 2022 10:06AM,238322,15,VT80173033,"Virtus Pharmaceuticals, LLC",Released
7,May 31 2022 10:06AM,238322,15,VT80173034,"Virtus Pharmaceuticals, LLC",Released
8,May 31 2022 10:06AM,238322,15,VT80173035,"Virtus Pharmaceuticals, LLC",Released
9,May 31 2022 10:06AM,238322,15,VT80173036,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,238389,Released,52
44,238390,Released,1
45,238393,Released,1
46,238394,Released,1
47,238395,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238389,NaN,NaN,52.0
238390,NaN,NaN,1.0
238393,NaN,NaN,1.0
238394,NaN,NaN,1.0
238395,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238288,0.0,0.0,41.0
238290,0.0,0.0,12.0
238316,0.0,0.0,8.0
238319,0.0,0.0,1.0
238322,0.0,0.0,19.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238288,0,0,41
238290,0,0,12
238316,0,0,8
238319,0,0,1
238322,0,0,19


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238288,0,0,41
1,238290,0,0,12
2,238316,0,0,8
3,238319,0,0,1
4,238322,0,0,19


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238288,,,41
1,238290,,,12
2,238316,,,8
3,238319,,,1
4,238322,,,19


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 31 2022 10:07AM,238395,18,"ClearSpec, LLC"
1,May 31 2022 10:07AM,238393,18,"Miracle Fruit Oil, LLC"
2,May 31 2022 10:07AM,238394,19,Eli Lilly and Company
3,May 31 2022 10:06AM,238322,15,"Virtus Pharmaceuticals, LLC"
22,May 31 2022 10:03AM,238389,10,ISDIN Corporation
74,May 31 2022 10:02AM,238319,15,"Carwin Pharmaceutical Associates, LLC"
75,May 31 2022 10:00AM,238388,10,ISDIN Corporation
121,May 31 2022 9:59AM,238390,10,ISDIN Corporation
122,May 31 2022 9:58AM,238316,15,"Person & Covey, Inc."
130,May 31 2022 9:55AM,238288,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 31 2022 10:07AM,238395,18,"ClearSpec, LLC",,,1
1,May 31 2022 10:07AM,238393,18,"Miracle Fruit Oil, LLC",,,1
2,May 31 2022 10:07AM,238394,19,Eli Lilly and Company,,,1
3,May 31 2022 10:06AM,238322,15,"Virtus Pharmaceuticals, LLC",,,19
4,May 31 2022 10:03AM,238389,10,ISDIN Corporation,,,52
5,May 31 2022 10:02AM,238319,15,"Carwin Pharmaceutical Associates, LLC",,,1
6,May 31 2022 10:00AM,238388,10,ISDIN Corporation,,,46
7,May 31 2022 9:59AM,238390,10,ISDIN Corporation,,,1
8,May 31 2022 9:58AM,238316,15,"Person & Covey, Inc.",,,8
9,May 31 2022 9:55AM,238288,15,"Brookfield Pharmaceuticals, LLC",,,41


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 10:07AM,238395,18,"ClearSpec, LLC",1,,
1,May 31 2022 10:07AM,238393,18,"Miracle Fruit Oil, LLC",1,,
2,May 31 2022 10:07AM,238394,19,Eli Lilly and Company,1,,
3,May 31 2022 10:06AM,238322,15,"Virtus Pharmaceuticals, LLC",19,,
4,May 31 2022 10:03AM,238389,10,ISDIN Corporation,52,,
5,May 31 2022 10:02AM,238319,15,"Carwin Pharmaceutical Associates, LLC",1,,
6,May 31 2022 10:00AM,238388,10,ISDIN Corporation,46,,
7,May 31 2022 9:59AM,238390,10,ISDIN Corporation,1,,
8,May 31 2022 9:58AM,238316,15,"Person & Covey, Inc.",8,,
9,May 31 2022 9:55AM,238288,15,"Brookfield Pharmaceuticals, LLC",41,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 10:07AM,238395,18,"ClearSpec, LLC",1,,
1,May 31 2022 10:07AM,238393,18,"Miracle Fruit Oil, LLC",1,,
2,May 31 2022 10:07AM,238394,19,Eli Lilly and Company,1,,
3,May 31 2022 10:06AM,238322,15,"Virtus Pharmaceuticals, LLC",19,,
4,May 31 2022 10:03AM,238389,10,ISDIN Corporation,52,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 10:07AM,238395,18,"ClearSpec, LLC",1.0,NaN,NaN
1,May 31 2022 10:07AM,238393,18,"Miracle Fruit Oil, LLC",1.0,NaN,NaN
2,May 31 2022 10:07AM,238394,19,Eli Lilly and Company,1.0,NaN,NaN
3,May 31 2022 10:06AM,238322,15,"Virtus Pharmaceuticals, LLC",19.0,NaN,NaN
4,May 31 2022 10:03AM,238389,10,ISDIN Corporation,52.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 10:07AM,238395,18,"ClearSpec, LLC",1.0,0.0,0.0
1,May 31 2022 10:07AM,238393,18,"Miracle Fruit Oil, LLC",1.0,0.0,0.0
2,May 31 2022 10:07AM,238394,19,Eli Lilly and Company,1.0,0.0,0.0
3,May 31 2022 10:06AM,238322,15,"Virtus Pharmaceuticals, LLC",19.0,0.0,0.0
4,May 31 2022 10:03AM,238389,10,ISDIN Corporation,52.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4767443,579.0,10.0,0.0
102,238367,1.0,0.0,0.0
12,1430121,80.0,0.0,0.0
15,1429921,100.0,0.0,0.0
18,953545,4.0,0.0,0.0
19,953479,10.0,3.0,1.0
20,476669,14.0,1.0,0.0
22,476730,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4767443,579.0,10.0,0.0
1,102,238367,1.0,0.0,0.0
2,12,1430121,80.0,0.0,0.0
3,15,1429921,100.0,0.0,0.0
4,18,953545,4.0,0.0,0.0
5,19,953479,10.0,3.0,1.0
6,20,476669,14.0,1.0,0.0
7,22,476730,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,579.0,10.0,0.0
1,102,1.0,0.0,0.0
2,12,80.0,0.0,0.0
3,15,100.0,0.0,0.0
4,18,4.0,0.0,0.0
5,19,10.0,3.0,1.0
6,20,14.0,1.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,579.0
1,102,Released,1.0
2,12,Released,80.0
3,15,Released,100.0
4,18,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,18,19,20,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Executing,10.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0
Released,579.0,1.0,80.0,100.0,4.0,10.0,14.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,18,19,20,22
0,Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,10.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0
2,Released,579.0,1.0,80.0,100.0,4.0,10.0,14.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,18,19,20,22
0,Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,10.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0
2,Released,579.0,1.0,80.0,100.0,4.0,10.0,14.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()